In [3]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import OneHotEncoder

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split


In [ ]:
pwd

In [ ]:
df = pd.read_csv('/projects/18_property_vtmfid/shared/structured_data.csv')

df.head()

In [ ]:
df.shape

In [ ]:
df['GT_basement'].loc[df['GT_basement'] == 'N'] = 0

In [ ]:
# dropping the one entry with '.' as its value for frontage
df.dropna(inplace=True)
print(df.shape)
df.drop(labels=2754, axis=0, inplace=True)
print(df.shape)

Prepare the dataframe for Random Forest Classifier

In [ ]:
df_train_1000 = pd.read_csv('train1000.csv')

In [ ]:
df_train_1000.head()

In [ ]:
df_train_1000.shape

Load the addresses used for testing set from C1 classifier

In [ ]:
df_C1 = pd.read_fwf('C1.txt').iloc[1:,0]
df_C1.head()

In [ ]:
df_C1.shape

Manipulate the dataset to get the test dataset 

In [ ]:
df_C1 = pd.DataFrame(df_C1)
print(df_C1.columns)
df_C1.head()

In [ ]:
df_C1 = df_C1['0'].str.split('.jpg', expand=True)
df_C1.head()

In [ ]:
# get the test set addresses from DoF by performing a merge
df_test_500 = pd.merge(df, df_C1, left_on='Address_lookup', right_on=0, how='inner')

In [ ]:
idx_test = list(df_test_500.iloc[:, 0])

df_test_500 = df.iloc[idx_test, :]

df_test_500.head()

Training set addresses are total addresses substract the testing set. 

In [ ]:
df_train = pd.merge(df, df_train_1000, left_on='Address_lookup', right_on='Address', how='inner')

In [ ]:
df_train.head()

In [ ]:
idx_train = list(df_train.iloc[:, 0])

df_train_1000 = df.iloc[idx_train, :]

df_train_1000.head()

In [ ]:
df_train_1000.shape

## Have to drop the duplicate addresses in the training set

In [ ]:
df_train_1000.drop_duplicates(subset='Address_lookup', keep='last', inplace=True)

# export the training and testing sets

In [1]:
df_train_1000.to_csv('c1_train.csv')

NameError: name 'df_train_1000' is not defined

In [2]:
df_test_500.to_csv('c1_500_test.csv')

NameError: name 'df_test_500' is not defined

## train and test the structure data with random forest

In [4]:
df_train_1000 = pd.read_csv('c1_train.csv')

df_test_500 = pd.read_csv('c1_500_test.csv')

In [5]:
from sklearn.base import TransformerMixin
from sklearn.preprocessing import StandardScaler

class DummyEncoder(TransformerMixin):
    '''
    Custom scikit-learn transformer for data that converts categorical features with 
    k classes to (k-1) dummy features.
    '''
    def fit(self, X=None, Y=None):
        # Do nothing
        return self
    
    def transform(self, X):
        categories = ['GT_Bldg_Class', 'GT_Style', 'GT_wall','GT_cond', 
                      'GT_grade', 'GT_basement']
        categories = [c for c in categories if c in X.columns.values]
        X = pd.get_dummies(X, drop_first=True, columns=categories)
        return X
    
class CustomScaler(TransformerMixin):
    '''
    Custom scikit-learn transformer for data that standardizes numerical features 
    to be zero mean and unit variance.
    '''
    def fit(self, X=None, Y=None):
        # Do nothing
        # Fit is implemented in transform method instead
        return self
    
    def transform(self, X):                
        numerical = ['GT_story','GT_frontage']
        numerical = [c for c in numerical if c in X.columns.values]
        scaler = StandardScaler().fit(X.loc[:,numerical])
        X.loc[:,numerical] = scaler.transform(X.loc[:,numerical])
        return X

In [6]:
scaler = CustomScaler()
df_train_1000 = scaler.fit(df_train_1000).transform(df_train_1000)
# Encode categorical features
encoder = DummyEncoder()
df_train_1000 = encoder.fit(df_train_1000).transform(df_train_1000)
display(df_train_1000)

,Unnamed: 0,Unnamed: 0.1,Address_lookup,GT_story,GT_frontage,GT_proximity,GT_Bldg_Class_A1,GT_Bldg_Class_A2,GT_Bldg_Class_A3,GT_Bldg_Class_A5,...,GT_grade_B+,GT_grade_B-,GT_grade_C,GT_grade_C+,GT_grade_C-,GT_grade_D,GT_grade_E,GT_grade_X,GT_basement_3,GT_basement_N
0,4,4,"175 MANHATTAN STREET, Staten Island, New York,...",-0.316949,0.017195,Detached,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,5,5,"171 MANHATTAN STREET, Staten Island, New York,...",-0.316949,0.017195,Detached,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,9,9,"149 MANHATTAN STREET, Staten Island, New York,...",-0.316949,-0.849789,Detached,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0
3,10,10,"145 MANHATTAN STREET, Staten Island, New York,...",-0.316949,-0.705292,Detached,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0
4,11,11,"141 MANHATTAN STREET, Staten Island, New York,...",-0.973612,-0.199551,Detached,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0
5,12,12,"64 PITTSVILLE AVENUE, Staten Island, New York,...",-0.973612,0.450688,Detached,0,1,0,0,...,0,0,1,0,0,0,0,0,1,0
6,13,13,"148 MANHATTAN STREET, Staten Island, New York,...",-2.306837,-0.416297,Detached,0,1,0,0,...,0,0,1,0,0,0,0,0,0,0
7,15,15,"164 MANHATTAN STREET, Staten Island, New York,...",-0.316949,0.522936,Detached,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
8,17,17,"178 MANHATTAN STREET, Staten Island, New York,...",0.677994,-0.849789,Detached,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
9,18,18,"180 MANHATTAN STREET, Staten Island, New York,...",-0.814421,-0.849789,Detached,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [7]:
scaler = CustomScaler()
df_test_500 = scaler.fit(df_test_500).transform(df_test_500)
# Encode categorical features
encoder = DummyEncoder()
df_test_500 = encoder.fit(df_test_500).transform(df_test_500)
display(df_test_500)

,Unnamed: 0,Unnamed: 0.1,Address_lookup,GT_story,GT_frontage,GT_proximity,GT_Bldg_Class_A1,GT_Bldg_Class_A2,GT_Bldg_Class_A3,GT_Bldg_Class_A5,...,GT_grade_A-,GT_grade_B,GT_grade_B+,GT_grade_B-,GT_grade_C,GT_grade_C+,GT_grade_D,GT_grade_X,GT_basement_3,GT_basement_N
0,1,1,"189 MANHATTAN STREET, Staten Island, New York,...",-2.049634,-0.423044,Detached,0,1,0,0,...,0,0,0,0,1,0,0,0,1,0
1,2051,2051,"100 MANEE AVENUE, Staten Island, New York, 10309",-2.049634,-0.345620,Detached,0,1,0,0,...,0,0,0,0,1,0,0,0,1,0
2,2055,2055,"73 MANEE AVENUE, Staten Island, New York, 10309",-0.234514,-0.190772,Detached,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,8,8,"155 MANHATTAN STREET, Staten Island, New York,...",-0.234514,0.428619,Detached,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,2057,2057,"65 MANEE AVENUE, Staten Island, New York, 10309",-0.234514,-0.190772,Detached,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
5,2061,2061,"68 BAYVIEW AVENUE, Staten Island, New York, 10309",0.673047,-0.887588,Detached,1,0,0,0,...,0,0,0,0,1,0,0,0,1,0
6,14,14,"158 MANHATTAN STREET, Staten Island, New York,...",-0.234514,-0.113349,Detached,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
7,2070,2070,"116 BAYVIEW AVENUE, Staten Island, New York, 1...",0.673047,-0.887588,Detached,1,0,0,0,...,0,0,0,0,1,0,0,0,1,0
8,2072,2072,"39 MANEE AVENUE, Staten Island, New York, 10309",-0.234514,1.125434,Detached,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
9,2076,2076,"5976 AMBOY ROAD, Staten Island, New York, 10309",0.673047,-0.423044,Detached,1,0,0,0,...,0,0,0,0,1,0,0,0,1,0


In [8]:
test_var = set(df_test_500.columns)
train_var = set(df_train_1000.columns)
diff_var = list(train_var - test_var)

In [9]:
df_test_500 = df_test_500[['Unnamed: 0',
 'Address_lookup',
 'GT_story',
 'GT_frontage',
 'GT_Bldg_Class_A1',
 'GT_Bldg_Class_A2',
 'GT_Bldg_Class_A3',
 'GT_Bldg_Class_A5',
 'GT_Bldg_Class_A9',
 'GT_Bldg_Class_B1',
 'GT_Bldg_Class_B2',
 'GT_Bldg_Class_B3',
 'GT_Bldg_Class_B9',
 'GT_Style_2.0',
 'GT_Style_3.0',
 'GT_Style_4.0',
 'GT_Style_6.0',
 'GT_Style_7.0',
 'GT_Style_8.0',
 'GT_Style_9.0',
 'GT_Style_10.0',
 'GT_Style_11.0',
 'GT_Style_14.0',
 'GT_wall_2.0',
 'GT_wall_3.0',
 'GT_wall_4.0',
 'GT_wall_5.0',
 'GT_wall_6.0',
 'GT_wall_7.0',
 'GT_wall_8.0',
 'GT_wall_9.0',
 'GT_cond_1',
 'GT_cond_2',
 'GT_cond_3',
 'GT_cond_4',
 'GT_cond_5',
 'GT_cond_6',
 'GT_cond_N',
 'GT_grade_A-',
 'GT_grade_B',
 'GT_grade_B+',
 'GT_grade_B-',
 'GT_grade_C',
 'GT_grade_C+',
 'GT_grade_D',
 'GT_grade_X',
 'GT_basement_3',
 'GT_basement_N',
'GT_proximity']]

In [10]:
df_train_1000.drop(columns=diff_var, inplace=True)

In [ ]:
list(df_train_1000.columns)

In [11]:
df_train_1000 = df_train_1000[['Unnamed: 0',
 'Address_lookup',
 'GT_story',
 'GT_frontage',
 'GT_Bldg_Class_A1',
 'GT_Bldg_Class_A2',
 'GT_Bldg_Class_A3',
 'GT_Bldg_Class_A5',
 'GT_Bldg_Class_A9',
 'GT_Bldg_Class_B1',
 'GT_Bldg_Class_B2',
 'GT_Bldg_Class_B3',
 'GT_Bldg_Class_B9',
 'GT_Style_2.0',
 'GT_Style_3.0',
 'GT_Style_4.0',
 'GT_Style_6.0',
 'GT_Style_7.0',
 'GT_Style_8.0',
 'GT_Style_9.0',
 'GT_Style_10.0',
 'GT_Style_11.0',
 'GT_Style_14.0',
 'GT_wall_2.0',
 'GT_wall_3.0',
 'GT_wall_4.0',
 'GT_wall_5.0',
 'GT_wall_6.0',
 'GT_wall_7.0',
 'GT_wall_8.0',
 'GT_wall_9.0',
 'GT_cond_2',
 'GT_cond_3',
 'GT_cond_4',
 'GT_cond_5',
 'GT_cond_6',
 'GT_cond_N',
 'GT_grade_A-',
 'GT_grade_B',
 'GT_grade_B+',
 'GT_grade_B-',
 'GT_grade_C',
 'GT_grade_C+',
 'GT_grade_D',
 'GT_grade_X',
 'GT_basement_3',
 'GT_basement_N','GT_proximity']]

In [12]:
df_test_500 = df_test_500[['Unnamed: 0',
 'Address_lookup',
 'GT_story',
 'GT_frontage',
 'GT_Bldg_Class_A1',
 'GT_Bldg_Class_A2',
 'GT_Bldg_Class_A3',
 'GT_Bldg_Class_A5',
 'GT_Bldg_Class_A9',
 'GT_Bldg_Class_B1',
 'GT_Bldg_Class_B2',
 'GT_Bldg_Class_B3',
 'GT_Bldg_Class_B9',
 'GT_Style_2.0',
 'GT_Style_3.0',
 'GT_Style_4.0',
 'GT_Style_6.0',
 'GT_Style_7.0',
 'GT_Style_8.0',
 'GT_Style_9.0',
 'GT_Style_10.0',
 'GT_Style_11.0',
 'GT_Style_14.0',
 'GT_wall_2.0',
 'GT_wall_3.0',
 'GT_wall_4.0',
 'GT_wall_5.0',
 'GT_wall_6.0',
 'GT_wall_7.0',
 'GT_wall_8.0',
 'GT_wall_9.0',
 'GT_cond_2',
 'GT_cond_3',
 'GT_cond_4',
 'GT_cond_5',
 'GT_cond_6',
 'GT_cond_N',
 'GT_grade_A-',
 'GT_grade_B',
 'GT_grade_B+',
 'GT_grade_B-',
 'GT_grade_C',
 'GT_grade_C+',
 'GT_grade_D',
 'GT_grade_X',
 'GT_basement_3',
 'GT_basement_N','GT_proximity']]

In [ ]:
OS=[]
for i in range(10):
    X_train, X_test, y_train, y_test = train_test_split(df.iloc[:,2:-2], df.iloc[:,-2:], test_size = 0.3, random_state = i)    
    dt = DecisionTreeClassifier()
    dt.fit(X_train,y_train)
    OS.append(dt.score(X_test,y_test))
print (np.mean(OS))

In [48]:
df_train_1000.GT_proximity

0      Detached
1      Detached
2      Detached
3      Detached
4      Detached
5      Detached
6      Detached
7      Detached
8      Detached
9      Detached
10     Detached
11     Detached
12     Detached
13     Detached
14     Detached
15     Detached
16     Detached
17     Detached
18     Detached
19     Detached
20     Detached
21     Detached
22     Detached
23     Detached
24     Detached
25     Detached
26     Detached
27     Detached
28     Detached
29     Detached
         ...   
970    Detached
971    Detached
972    Detached
973    Detached
974    Detached
975    Detached
976    Detached
977    Detached
978    Detached
979    Detached
980    Detached
981    Detached
982    Detached
983    Detached
984    Detached
985    Detached
986    Detached
987    Detached
988    Detached
989    Detached
990    Detached
991    Detached
992    Detached
993    Detached
994    Detached
995    Detached
996    Detached
997    Detached
998    Detached
999    Detached
Name: GT_proximity, Leng

In [23]:
X_train = df_train_1000.iloc[: , 2: -1]
y_train = df_train_1000.iloc[:, -1]

X_test = df_test_500.iloc[: , 2: -1]
y_test = df_test_500.iloc[:, -1]


In [ ]:
df_test_500.head()

In [24]:
param_grid ={'max_depth':range(1,11)}
rf = RandomForestClassifier(n_estimators=100)
gr = GridSearchCV(rf, param_grid=param_grid)
rs = gr.fit(X_train, y_train)
print ('Best parameter value:',rs.best_params_)
y_predict = rs.predict(X_test)
print('Accuracy = ',(pd.Series((y_predict == y_test)).value_counts(normalize=True)[True]))

('Best parameter value:', {'max_depth': 3})
('Accuracy = ', 0.90400000000000003)


In [17]:
# In what order the probability for the classes are presented
rs.classes_

array(['Attached', 'Detached', 'Semi-Attached'], dtype=object)

In [22]:
y_test.shape

(1000,)

In [27]:
pred_class = rs.predict_proba(X_test)
print(pred_class)

[[ 0.08370817  0.85362674  0.06266509]
 [ 0.0966185   0.8405452   0.06283631]
 [ 0.06847062  0.89035977  0.04116961]
 ..., 
 [ 0.06786256  0.88414744  0.04799   ]
 [ 0.07442415  0.86643824  0.05913762]
 [ 0.05485645  0.9009089   0.04423465]]


In [30]:
np.array(pred_class).shape

(500, 3)

In [36]:
pred_class = np.array(pred_class).T

In [40]:
output = {'Address':df_test_500.iloc[:, 1], 'Attached':pred_class[0], 'Detached':pred_class[1], 'Semi-attached':pred_class[2]}

In [41]:
df_output = pd.DataFrame(output)

In [42]:
df_output.head()

,Address,Attached,Detached,Semi-attached
0,"189 MANHATTAN STREET, Staten Island, New York,...",0.083708,0.853627,0.062665
1,"100 MANEE AVENUE, Staten Island, New York, 10309",0.096618,0.840545,0.062836
2,"73 MANEE AVENUE, Staten Island, New York, 10309",0.068471,0.890360,0.041170
3,"155 MANHATTAN STREET, Staten Island, New York,...",0.075021,0.883634,0.041344
4,"65 MANEE AVENUE, Staten Island, New York, 10309",0.068471,0.890360,0.041170


In [47]:
df_output.describe()

,Attached,Detached,Semi-attached
count,500.000000,500.000000,500.000000
mean,0.239489,0.672726,0.087785
std,0.252306,0.299579,0.058138
min,0.030052,0.061895,0.025653
25%,0.059161,0.356154,0.043488
50%,0.094461,0.842493,0.061880
75%,0.420808,0.897371,0.123810
max,0.832023,0.943561,0.281331


In [43]:
df_output.to_csv('DOF_rf_output.csv')

In [50]:
Feature_importance=pd.DataFrame([list(X_train.columns),list(rs.feature_importances_)]).T
Feature_importance.columns=["variables","importance"]

# list the top 5 most important features in order
Feature_importance.sort_values(by="importance",ascending=False).iloc[:5,:]

AttributeError: 'GridSearchCV' object has no attribute 'feature_importances_'